In [ ]:
#!pip install --upgrade transformers==3.0.2 #Make sure of transformer version
#!pip install git+https://github.com/huggingface/transformers
import pandas as pd
from simpletransformers.t5 import T5Model
from sklearn.model_selection import train_test_split
import logging
import numpy as np

logging.basicConfig(level=logging.ERROR)
from sklearn.metrics import accuracy_score

import pandas as pd
from simpletransformers.t5 import T5Model

In [ ]:
prefix = 'data/binary_classification/'

binary_train_df = pd.read_csv(prefix + 'train.csv', header=None)
binary_train_df.head()

binary_eval_df = pd.read_csv(prefix + 'test.csv', header=None)
binary_eval_df.head(1)

binary_train_df[0] = (binary_train_df[0] == 2).astype(int)
binary_eval_df[0] = (binary_eval_df[0] == 2).astype(int)

binary_train_df = pd.DataFrame({
    'prefix': ["binary classification" for i in range(len(binary_train_df))],
    'input_text': binary_train_df[1].str.replace('\n', ' '),
    'target_text': binary_train_df[0].astype(str),
})

print(binary_train_df.head())

binary_eval_df = pd.DataFrame({
    'prefix': ["binary classification" for i in range(len(binary_eval_df))],
    'input_text': binary_eval_df[1].str.replace('\n', ' '),
    'target_text': binary_eval_df[0].astype(str),
})


print(binary_eval_df.head(1))
print ( binary_train_df.shape, binary_eval_df.shape)

In [ ]:
binary_eval_df.iloc[1, 1]

In [4]:
train_df = binary_train_df.copy()
eval_df = binary_eval_df.copy()

In [5]:
train_df.to_csv("data/train.tsv", "\t", index=False)
eval_df.to_csv("data/eval.tsv", "\t", index=False)

In [6]:
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/eval.tsv", sep="\t").astype(str)

In [7]:
train_df.head()

,prefix,input_text,target_text
0,binary classification,"Unfortunately, the frustration of being Dr. Go...",0
1,binary classification,Been going to Dr. Goldberg for over 10 years. ...,1
2,binary classification,I don't know what Dr. Goldberg was like before...,0
3,binary classification,I'm writing this review to give you a heads up...,0
4,binary classification,All the food is great here. But the best thing...,1


In [8]:


model_args = {
    "max_seq_length": 128,
    "train_batch_size": 16,
    "eval_batch_size": 64,
    "num_train_epochs": 2,
#     "evaluate_during_training": True,
#     "evaluate_during_training_steps": 15000,
#     "evaluate_during_training_verbose": True,
    
    "use_multiprocessing": False,
    "fp16": False,

    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,

    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    'n_gpu': 4,
    'process_count': 30

    #"wandb_project": "T5 mixed tasks - Binary, Multi-Label, Regression"
}

model = T5Model("t5-base", args=model_args)



In [ ]:
#model.train_model (train_df.head(100000), eval_data= eval_df.head(200))
model.train_model (train_df.head(120000))

In [ ]:
model.model.save_pretrained('outputs/t5kg')
model.tokenizer.save_pretrained('outputs/t5kg')

In [ ]:
model = T5Model('outputs/t5kg', args=model_args)

In [ ]:
model.predict(['binary classification: this absolutely worst', 
               'binary classification: this absolutely great',
                'I am so happy but movie was just ok. ',
                'the movie was bad'])

In [ ]:
pred = model.predict(eval_df['prefix'].head(10000) +': '+ eval_df['input_text'].head(10000) )

In [ ]:
set(pred)

In [ ]:
#pred.count('entailment')
# pred = ['0' if x=='positive' else x for x in pred]
# pred = ['0' if x== 'entailment' else x for x in pred]

In [ ]:
pred = list (map( int, pred))

In [ ]:
y = eval_df['target_text'].values[:5000]
y = list (map( int, y))

In [ ]:

pd.crosstab(np.array(y),np.array(pred) )

In [ ]:
accuracy_score(np.array(y),np.array(pred))